# Create Cluster: RDB
This notebook will create (start) an RDB cluster.

In [1]:
%%html
<style>
table {float:left}
</style>

## Setup

### Node Types
|Type|Mem (GB)|vCPUs|
|:---|---:|---:|
|kx.s.large|12|2|
|kx.s.xlarge|27|4|
|kx.s.2xlarge|54|8|
|kx.s.4xlarge|108|16|
|kx.s.8xlarge|216|32|
|kx.s.16xlarge|432|64|
|kx.s.32xlarge|864|128|


In [2]:
import os
import boto3
import json
import datetime

from managed_kx import *
from env_2 import *

# Managed KX Database and Cluster names to create
DB_NAME="welcomedb"

SEC_THREADS='8'
CLUSTER_NAME=f"RDB_{DB_NAME}"

# Cluster Settings
CODEBASE="code"
S3_CODE_PATH="code"

CAPACITY_CONFIG={ 'nodeCount': 1, 'nodeType': 'kx.s.2xlarge'}
SAVEDOWN_CONFIG={ 'type': 'SDS01', 'size':200 }
DATABASE_CONFIG=[{ 'databaseName': DB_NAME }]
CODE_CONFIG={ 's3Bucket': S3_BUCKET, 's3Key': f'{S3_CODE_PATH}/{CODEBASE}.zip' }

INIT_SCRIPT=f'{CODEBASE}/init.q'
CMD_ARGS=[
    { 'key': 's', 'value': SEC_THREADS }, 
    { 'key': 'dbname', 'value': DB_NAME}, 
    { 'key': 'codebase', 'value': CODEBASE} 
]

In [3]:
# triggers credential get
session=None

try:
    subprocess.call(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS sessio: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using variables ...


## Create Cluster

In [4]:
resp = client.create_kx_cluster(
    environmentId=ENV_ID, 
    clusterName=CLUSTER_NAME,
    clusterType='RDB',
    releaseLabel = '1.0',
    databases=DATABASE_CONFIG,
    capacityConfiguration=CAPACITY_CONFIG,
    savedownStorageConfiguration=SAVEDOWN_CONFIG,
    clusterDescription="Created with create_cluster_RDB notebook",
    code=CODE_CONFIG,
    initializationScript=INIT_SCRIPT,
    azMode=AZ_MODE,
    availabilityZoneId=AZ_ID,
    commandLineArguments=CMD_ARGS,
    vpcConfiguration={ 
        'vpcId': VPC_ID,
        'securityGroupIds': SECURITY_GROUPS,
        'subnetIds': SUBNET_IDS,
        'ipAddressType': 'IP_V4' }
)

In [5]:
wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=CLUSTER_NAME, show_wait=True)
print()
print("** DONE **")

Cluster: RDB_welcomedb status is PENDING, total wait 0:00:00, waiting 30 sec ...
Cluster: RDB_welcomedb status is CREATING, total wait 0:00:30, waiting 30 sec ...
Cluster: RDB_welcomedb status is CREATING, total wait 0:01:00, waiting 30 sec ...
Cluster: RDB_welcomedb status is CREATING, total wait 0:01:30, waiting 30 sec ...
Cluster: RDB_welcomedb status is CREATING, total wait 0:02:00, waiting 30 sec ...
Cluster: RDB_welcomedb status is CREATING, total wait 0:02:30, waiting 30 sec ...
Cluster: RDB_welcomedb status is CREATING, total wait 0:03:00, waiting 30 sec ...
Cluster: RDB_welcomedb status is CREATING, total wait 0:03:30, waiting 30 sec ...
Cluster: RDB_welcomedb status is CREATING, total wait 0:04:00, waiting 30 sec ...
Cluster: RDB_welcomedb status is CREATING, total wait 0:04:30, waiting 30 sec ...
Cluster: RDB_welcomedb status is CREATING, total wait 0:05:00, waiting 30 sec ...
Cluster: RDB_welcomedb status is CREATING, total wait 0:05:30, waiting 30 sec ...
Cluster: RDB_welc

In [ ]:
# Give permissions time to propogate after cluster creation....
time.sleep(60)

In [7]:
conn_str = get_kx_connection_string(client, environmentId=ENV_ID, clusterName=CLUSTER_NAME, userName=KDB_USERNAME, boto_session=session)
print(conn_str)

:tcps://vpce-0a299a1f762f351b0-uiaa28cm.vpce-svc-061a7d8630e15dd1a.us-east-1.vpce.amazonaws.com:443:bob:Host=vpce-0a299a1f762f351b0-uiaa28cm.vpce-svc-061a7d8630e15dd1a.us-east-1.vpce.amazonaws.com&Port=5000&User=bob&Action=finspace%3AConnectKxCluster&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEP3%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJHMEUCIQCgBdnCSb7VbDcOBBMHmptBNq4UEXM707K0Vz2%2FyuOBbwIgXHOGKdMrs5Pd29AotL3djT65Aoi2Ft5It3QBSxaAjewq%2BAIIRhAAGgw2MTI4NDEzODM1OTQiDFSliLTQPhEn0U%2FunirVAnLKiKasSNUQdclq2RvDDJyf%2BdNe4fYEHwsyBL%2BNcDsCOkgLrXAk3DXDI%2FrhGpB6Jwsir8p8H2HGqye%2B4vWY8o6x8lGwKLbO60AhoFYe7OZVfS0eUOPr5it2p0hj58amb3jvhLVPCB%2FQejQJaaYtgpDlklQZfA2VpMXtArl%2B9rzN3C8NQiOQokuZidg6%2Fc8Y3DYacZ84D13jkCbgc7yaBjS%2BJeESE%2BBBWuIzBq8eFOYhE%2F0KMBc2A2EGo4pQt2ly6U4Kbs6Y7v97h9Wz6wVBmZTYE2E6nrwuneFG%2B9PMTb7yNYtyQc8S%2F4%2BWZd6cJ89lQgURN5WWv1D8Fb0%2BHffaSh4%2BTpuGyRyiWJSMEJYAfGWfbNSV6jRR%2BXM%2BaV1ZDa6nPzXiTggQCKeIceAE0wqdoO1i4ixD3S%2Fo2feYbhQQwm%2BmV1kSWsYAU7mkArVuoll4wsLp1USsMLnc%2FKMGOr8B2E

In [8]:
cdf = get_clusters(client, ENV_ID)

display(cdf)

clusterName,status,clusterType,capacityConfiguration,commandLineArguments,clusterDescription,lastModifiedTimestamp,createdTimestamp,databaseName,cacheConfigurations
HDB_TAQ_2021_2D,RUNNING,HDB,"{'nodeType': 'kx.s.32xlarge', 'nodeCount': 2}","[{'key': 's', 'value': '8'}, {'key': 'dbname', 'value': 'TAQ_2021_2D'}, {'key': 'codebase', 'value': 'taqcode'}]",Created with create_cluster_TAQ_2D notebook,2023-06-06 12:54:08.985000+00:00,2023-06-06 12:40:07.108000+00:00,TAQ_2021_2D,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/2021.01.04/', '/2021.01.05/']}]"
HDB_basictickdb_20230601,RUNNING,HDB,"{'nodeType': 'kx.s.xlarge', 'nodeCount': 2}","[{'key': 's', 'value': '8'}, {'key': 'dbname', 'value': 'basictickdb'}, {'key': 'codebase', 'value': 'basictick'}]",Created with create_HDB for basic_tick notebook,2023-06-06 11:52:52.544000+00:00,2023-06-06 11:35:14.234000+00:00,basictickdb,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}]"
HDB_welcomedb,RUNNING,HDB,"{'nodeType': 'kx.s.2xlarge', 'nodeCount': 3}","[{'key': 's', 'value': '4'}, {'key': 'dbname', 'value': 'welcomedb'}, {'key': 'codebase', 'value': 'code'}]",Created with create_cluster_HDB notebook,2023-06-06 11:50:28.453000+00:00,2023-06-06 11:32:13.700000+00:00,welcomedb,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}]"
RDB_basictickdb_20230601,RUNNING,RDB,"{'nodeType': 'kx.s.2xlarge', 'nodeCount': 1}","[{'key': 's', 'value': '8'}, {'key': 'dbname', 'value': 'basictickdb'}, {'key': 'codebase', 'value': 'basictick'}, {'key': 'tphostfile', 'value': 'tickerplant2'}]",Created with create_RDB notebook,2023-06-06 11:54:07.399000+00:00,2023-06-06 11:38:47.020000+00:00,basictickdb,[]
RDB_welcomedb,RUNNING,RDB,"{'nodeType': 'kx.s.2xlarge', 'nodeCount': 1}","[{'key': 's', 'value': '8'}, {'key': 'dbname', 'value': 'welcomedb'}, {'key': 'codebase', 'value': 'code'}]",Created with create_cluster_RDB notebook,2023-06-06 11:47:08.448000+00:00,2023-06-06 11:32:06.677000+00:00,welcomedb,[]
cluster_create_delete_db_20230606_1132,RUNNING,HDB,"{'nodeType': 'kx.s.xlarge', 'nodeCount': 2}","[{'key': 's', 'value': '4'}, {'key': 'dbname', 'value': 'create_delete_db_20230606_1132'}, {'key': 'codebase', 'value': 'code'}]",Demo Cluster for database create_delete_db_20230606_1132,2023-06-06 11:50:54.875000+00:00,2023-06-06 11:33:18.188000+00:00,create_delete_db_20230606_1132,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}]"
hdb-cluster-welcomedb,RUNNING,HDB,"{'nodeType': 'kx.s.xlarge', 'nodeCount': 2}","[{'key': 's', 'value': '4'}, {'key': 'dbname', 'value': 'welcomedb'}, {'key': 'codebase', 'value': 'code'}]",None,2023-06-06 11:44:19.914000+00:00,2023-06-06 11:26:51.517000+00:00,welcomedb,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}]"


In [9]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2023-06-06 13:01:48.118482
